1. Grid - https://www.w3schools.com/css/css_grid.asp 
2. Popup - https://www.w3schools.com/howto/howto_js_popup_form.asp
3. Progress Bar - https://www.w3schools.com/tags/tag_progress.asp
4. None - Does not belong either of the above

In [3]:
%matplotlib inline
import pandas as pd
import torch, copy, json, requests, os, time, cv2, glob, random, shutil
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from io import BytesIO
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import datasets, models, transforms
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import f1_score, recall_score
from src import *
%load_ext autoreload
%autoreload 2
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)
base_path = './data/images/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Train test split image folder

In [4]:
train_dir = os.path.join(base_path, 'train')
val_dir = os.path.join(base_path, 'val')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for cat in ['grid', 'popup', 'progress_bar', 'none']:
    image_ls = glob.glob(base_path + cat + '/*')
    train_img_ls = random.sample(image_ls, len(image_ls) // 2)
    val_image_ls = list(set(image_ls) - set(train_img_ls))
    os.makedirs(os.path.join(train_dir, cat), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cat), exist_ok=True)
    for img_path in train_img_ls:
        shutil.copy(img_path, os.path.join(train_dir, cat, os.path.basename(img_path)))
    for img_path in val_image_ls:
        shutil.copy(img_path, os.path.join(val_dir, cat, os.path.basename(img_path)))

### Data Transformations

In [5]:
base_transform = [transforms.Resize((224, 224)),  
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]

grid_transform = transforms.Compose([ transforms.ColorJitter(brightness=0.2, contrast=0.2),
                                        transforms.RandomAffine(degrees=10, translate=(0.05, 0.05))] + 
                                        base_transform)
popup_transform = transforms.Compose([ transforms.ColorJitter(brightness=0.2, contrast=0.2),
                                         transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),  
                                        transforms.RandomHorizontalFlip()] + 
                                        base_transform)
progress_bar_transform = transforms.Compose([ transforms.ColorJitter(brightness=0.2, contrast=0.2),
                                        transforms.Lambda(pad_to_square)] + 
                                        base_transform)

none_transform = transforms.Compose([ transforms.Lambda(pad_to_square),
                                        transforms.ColorJitter(brightness=0.2, contrast=0.2), 
                                        transforms.RandomAffine(degrees=5, translate=(0.05, 0.05))] +
                                        base_transform)

val_transform   = transforms.Compose([  transforms.Lambda(pad_to_square)] +
                                        base_transform)
class_transforms = {'grid': grid_transform, 
                    'popup': popup_transform,
                    'progress_bar': progress_bar_transform,
                    'none': none_transform}

In [6]:
class CustomImageFolder(datasets.ImageFolder):
    def __init__(self, root, class_transforms, transform=None):
        super(CustomImageFolder, self).__init__(root, transform)
        self.class_transforms = class_transforms
        self.class_to_label_name = {0: 'grid',            
                                    1: 'none',
                                    2: 'popup',
                                    3: 'progress_bar'}
    
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = self.loader(path)
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        class_name = self.class_to_label_name[target]
        if self.transform is None and class_name in self.class_transforms:
            img = self.class_transforms[class_name](img)
        else:
            img = self.transform(img)
            
        return img, target

In [7]:
train_dataset = CustomImageFolder(root=f'{base_path}/train', class_transforms=class_transforms)
val_dataset = CustomImageFolder(root=f'{base_path}/val', class_transforms=None, transform = val_transform)

In [8]:
len(train_dataset), len(val_dataset)

(852, 855)

In [9]:
dataloaders = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True),
              'val': torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)}

In [11]:
train_dataset.class_to_idx

{'grid': 0, 'none': 1, 'popup': 2, 'progress_bar': 3}

In [12]:
class_names = train_dataset.classes
class_names

['grid', 'none', 'popup', 'progress_bar']

In [21]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names)) 
model = model.to(device)

/Users/patsnap/miniforge3/envs/tf_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/patsnap/miniforge3/envs/tf_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
image_ratio = [len(glob.glob(base_path+f'train/{cl_name}/*.png')) for cl_name in class_names]

weighted_ratio = [round(len(train_dataset)/image_num ,1) for image_num in image_ratio]

print('Image Num, class_weights',list(zip(image_ratio, weighted_ratio)))

class_weights = torch.tensor(weighted_ratio).to(device)  
criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = nn.CrossEntropyLoss()

# Define optimizer and learning rate scheduler
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

Image Num, class_weights [(368, 2.3), (354, 2.4), (42, 20.3), (88, 9.7)]


In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=200, output_csv='metrics.csv'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    metrics = []
    patience = 50
    patience_counter = 0  
    previous_lr = None
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []

            data_loader = tqdm(dataloaders[phase], desc=f'{phase.capitalize()} Epoch {epoch+1}/{num_epochs}')
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())


            data_loader.set_postfix({f'{phase.capitalize()} Loss': running_loss / len(data_loader)})

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset)
            f1 = f1_score(all_labels, all_preds, average='macro')  
            recall = recall_score(all_labels, all_preds, average='macro')  


            if phase == 'val':
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}, f1:{f1:.4f}, recall:{recall:.4f} ')
                metrics.append({'epoch': epoch,'phase': phase,'loss': epoch_loss,'accuracy': epoch_acc.item(),'f1_score': f1,'recall': recall})
                scheduler.step(epoch_loss)
                current_lr = optimizer.param_groups[0]['lr']
                if previous_lr is None or current_lr != previous_lr:
                    print(f'Learning rate changed: {current_lr:.6f}')
                    previous_lr = current_lr

                
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    print(f'epoch: {epoch + 1}, best_model_wts save')
                    
                    patience_counter = 0  
                else:
                    patience_counter += 1
                    
                if patience_counter >= patience:
                    print(f'Early stopping triggered after {epoch + 1} epochs without improvement.')
                    pd.DataFrame(metrics).to_csv(output_csv, index=False)
                    model.load_state_dict(best_model_wts)
                    return model  

    

    print(f'Best val loss: {best_loss:.4f}')

    model.load_state_dict(best_model_wts)
    pd.DataFrame(metrics).to_csv(output_csv, index=False)
    return model

In [24]:
# Train the model
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=200)

Train Epoch 1/200:   0%|                                                                           | 0/27 [00:00<?, ?it/s]/Users/patsnap/miniforge3/envs/tf_env/lib/python3.8/site-packages/torch/utils/_device.py:77: UserWarning: MPS: randperm op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Distributions.mm:392.)
  return func(*args, **kwargs)
Val Epoch 1/200: 100%|████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.09it/s]


val Loss: 0.6611 Acc: 0.7673, f1:0.5757, recall:0.6379 
Learning rate changed: 0.001000
epoch: 1, best_model_wts save


Val Epoch 2/200: 100%|████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.10it/s]


val Loss: 0.4834 Acc: 0.8140, f1:0.6339, recall:0.6620 
epoch: 2, best_model_wts save


Val Epoch 3/200: 100%|████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.11it/s]


...

val Loss: 0.3036 Acc: 0.8959, f1:0.8627, recall:0.9013 


Val Epoch 61/200: 100%|███████████████████████████████████████████████████████████████████| 27/27 [00:13<00:00,  2.07it/s]


val Loss: 0.3214 Acc: 0.8901, f1:0.8546, recall:0.8924 


Val Epoch 62/200: 100%|███████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.14it/s]

val Loss: 0.3077 Acc: 0.8936, f1:0.8566, recall:0.8946 
Early stopping triggered after 62 epochs without improvement.


In [25]:
# Save the model
torch.save(model.state_dict(), 'best_model_v3.pth')

### Partial Freezing on layer4

In [26]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names)) 

#fine tuned the last block
for name, param in model.named_parameters():
    if "layer4" in name:  
        param.requires_grad = True
model = model.to(device)

weighted_ratio = [round(len(train_dataset)/len(glob.glob(base_path+f'train/{cl_name}/*.png')),1) for cl_name in class_names]

class_weights = torch.tensor(weighted_ratio).to(device)  
criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion = nn.CrossEntropyLoss()

# Define optimizer and learning rate scheduler
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

/Users/patsnap/miniforge3/envs/tf_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/patsnap/miniforge3/envs/tf_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/patsnap/miniforge3/envs/tf_env/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [27]:
# Train the model
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=200, output_csv='metrics_layer4_v2.csv')

Val Epoch 1/200: 100%|████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.12it/s]


val Loss: 0.6461 Acc: 0.7848, f1:0.5986, recall:0.6452 
Learning rate changed: 0.001000
epoch: 1, best_model_wts save


Val Epoch 2/200: 100%|████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.13it/s]


val Loss: 0.5412 Acc: 0.7789, f1:0.6430, recall:0.6763 
epoch: 2, best_model_wts save


Val Epoch 3/200: 100%|████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.13it/s]


...

val Loss: 0.4295 Acc: 0.8550, f1:0.8046, recall:0.8606 


Val Epoch 59/200: 100%|███████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.13it/s]


val Loss: 0.4391 Acc: 0.8538, f1:0.8052, recall:0.8652 


Val Epoch 60/200: 100%|███████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.11it/s]

val Loss: 0.4261 Acc: 0.8596, f1:0.8070, recall:0.8634 
Early stopping triggered after 60 epochs without improvement.


In [28]:
# Save the model
torch.save(model.state_dict(), 'best_model_layer4_v2.pth')